In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617785 sha256=b88b89f43e01bfffbdcf32acc0f273e671114846a597e2e8645714f559de2ca5
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import io
import pandas as pd
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise.dataset import DatasetAutoFolds
from surprise import Trainset, Reader
from surprise.model_selection import cross_validate
from collections import defaultdict
import pickle
from surprise.dump import dump
from google.colab import files
import uuid
from sklearn.externals import joblib

def train_model(csv):
  # uploaded=files.upload()
  print('Carregando a base de dados')
  csv = pd.read_csv('df.csv')
  csv = Dataset.load_from_df(csv[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  print('Ok')
  algo = SVD(biased=False)
  cross_validate(algo, csv, cv=5)
  #gs = GridSearchCV(SVD, param_grid={'n_epochs': (3,5,10,50,100), 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}, cv=5, measures='rmse', refit=True)
  print('Criando o dataset de treino')
  trainset = csv.build_full_trainset()
  print('Ok')
  # remover depois
  # testset = trainset.build_testset()
  print('Iniciando o treinamento dos dados')
  modelo = algo.fit(trainset) #treino do algoritmo no dataset inteiro
  #params_treino = gs.best_params['rmse']
  #print('Melhores parâmetros: \t', gs.best_params['rmse'])
  print('Finalizado')
  return modelo
  
def save_model(model):
  file_name = str(uuid.uuid4()) + '.model'
  #file_name = 'Modelo final.pickle'
  #pickle.dump(model, open(file_name, 'wb'))
  pickle.dump(model, open(file_name, 'wb'))
  #joblib.dump(model, file_name)
  print(file_name)
  return file_name
 
def load_model(file_name):
  loaded_model = pickle.load(open(file_name, 'rb'))
  #loaded_model = joblib.load(file_name)
  return loaded_model
 
def predicoes(csv, model, uid, iid):
  model_loaded = load_model(file_name='Modelo final.pickle')
  csv = pd.read_csv('df.csv')
  csv = Dataset.load_from_df(csv[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  trainset = csv.build_full_trainset()
  testset = trainset.build_testset()
  predicoes = model_loaded.test(testset)
  pred = model_loaded.predict(uid, iid, verbose=True)
  uid = Trainset.to_raw_uid(iuid)
  iid = Trainset.to_raw_iid(iiid)
  #predicoes = pd.DataFrame(predicoes)
  return predicoes

def get_top_n(predicoes, n=5):
  top_n = defaultdict(list)
  for uid, iid, true_r, est, _ in predicoes:
    top_n[uid].append((iid,est)) #predições para cada usuário
  for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True) #classificação das previsões e seleção das mais altas
    top_n[uid] = user_ratings[:n]
  for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
  return top_n

In [ ]:
!pwd

/content


In [ ]:
print('Carregando a base de dados')
uploaded=files.upload()

Carregando a base de dados


Saving df.csv to df.csv


In [ ]:
# treino
%%time
csv = 'df.csv'
print('training model...')
model = train_model(csv)
print('saving model...')
model_filename = save_model(model)
print(f'model saved to {model_filename}')


training model...
Carregando a base de dados
Ok
Criando o dataset de treino
Ok
Iniciando o treinamento dos dados
Finalizado
saving model...
0e29b27a-4c99-4850-a958-fc1a8ee5c6c4.model
model saved to 0e29b27a-4c99-4850-a958-fc1a8ee5c6c4.model
CPU times: user 192 ms, sys: 2.45 ms, total: 194 ms
Wall time: 196 ms


In [ ]:
!ls

 0e29b27a-4c99-4850-a958-fc1a8ee5c6c4.model  'Modelo final.pickle'
 df.csv					      sample_data


In [ ]:
# predição
print('Carregando o modelo...')
model = load_model(model_filename)
print ("Modelo carregado")
iuid = str(2)
iiid = str(15)
#est = str(4.15)
print(f'predicting product {iid}..')
print('Realizando as predições')
predict = predicoes(csv, model, uid, iid)
top_n = get_top_n(predict)
print(f'prediction result={top_n}')
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
from surprise.prediction_algorithms import knns
from surprise import KNNBaseline

def train_model(csv):
  # uploaded=files.upload()
  print('Carregando a base de dados')
  csv = pd.read_csv('df.csv')
  csv = Dataset.load_from_df(csv[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  print('Ok')
  algo = SVD(biased=False)
  cross_validate(algo, csv, cv=5)
  #gs = GridSearchCV(SVD, param_grid={'n_epochs': (3,5,10,50,100), 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}, cv=5, measures='rmse', refit=True)
  print('Criando o dataset de treino')
  trainset = csv.build_full_trainset()
  print('Ok')
  # remover depois
  # testset = trainset.build_testset()
  print('Iniciando o treinamento dos dados')
  modelo = algo.fit(trainset) #treino do algoritmo no dataset inteiro
  #params_treino = gs.best_params['rmse']
  #print('Melhores parâmetros: \t', gs.best_params['rmse'])
  print('Finalizado')
  return modelo
  
def save_model(model):
  file_name = 'Modelo final.pickle'
  pickle.dump(model, open(file_name, 'wb'))
  file_name = str(uuid.uuid4()) + '.model'
  print(file_name)
  return file_name
 
def load_model(file_name):
  print('Carregando o modelo')
  loaded_model = pickle.load(open(file_name, 'rb'))
  print ("Modelo carregado")
  return loaded_model
 
def predicoes(csv, model, uid, iid):
  print('Realizando as predições')
  model_loaded = load_model(file_name='Modelo final.pickle')
  csv = pd.read_csv('df.csv')
  csv = Dataset.load_from_df(csv[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  trainset = csv.build_full_trainset()
  testset = trainset.build_anti_testset()
  predicoes = model_loaded.test(testset)
  pred = model_loaded.predict(uid, iid, verbose=True)
  #predicoes = pd.DataFrame(predicoes)
  return predicoes

def get_top_n(predicoes, n=5):
  top_n = defaultdict(list)
  for uid, iid, true_r, est, _ in predicoes:
    top_n[uid].append((iid,est)) #predições para cada usuário
  for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True) #classificação das previsões e seleção das mais altas
    top_n[uid] = user_ratings[:n]
  #for uid, user_ratings in top_n.items():
    #print(uid, [iid for (iid, _) in user_ratings])
  return top_n

# Ronivaldo Criou a partir daqui

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 46 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617793 sha256=1650b17eb2b3e2aa4adefc8fa1da466b90f4dda88c9989f0c203f071a1bdcd70
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import uuid
import sys
import os
import pickle
import time
from collections import defaultdict
from surprise.prediction_algorithms import knns
from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        if n:
          top_n[uid] = user_ratings[:n]
        else:
          top_n[uid] = user_ratings

    return top_n

def train_model(csv_filename, n=None):
  start = time.time()
  # TODO: alterar para ler .csv
  # print('loading from dataset')
  # df = pd.read_csv(csv_filename)
  # data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  data = Dataset.load_builtin('ml-100k')
  trainset = data.build_full_trainset()
  algo = SVD(random_state=1)
  algo.fit(trainset)

  # Than predict ratings for all pairs (u, i) that are NOT in the training set.
  testset = trainset.build_anti_testset()
  predictions = algo.test(testset)

  model = get_top_n(predictions, n=n)
  elapsed_seconds = time.time() - start
  print(f'model trained in {elapsed_seconds:.1f} seconds')
  return model

def save_model(model):
  tamanho_modelo = sys.getsizeof(model)
  print(f'tamanho do modelo: {tamanho_modelo} bytes')
  file_name = str(uuid.uuid4()) + '.model'
  pickle.dump(model, open(file_name, 'wb'))
  tamanho_arquivo = os.stat(file_name).st_size
  print(f'tamanho do arquivo: {tamanho_arquivo} bytes')
  return file_name

def load_model(file_name):
  print('Carregando o modelo')
  loaded_model = pickle.load(open(file_name, 'rb'))
  print ("Modelo carregado")
  return loaded_model

def predict_user(model, user_id, n=5):
  result = []
  # Print the recommended items for each user
  for uid, user_ratings in model.items():
    if str(uid) == str(user_id):
      result = [iid for (iid, _) in user_ratings][:n]
  return result

def predict_item(model, item_id, n=5):
  result = []
  # Print the recommended items for each user
  top_n_iid = defaultdict(list)
  for uid, user_ratings in model.items():
    for (iid, rating) in user_ratings:
      if str(iid) == str(item_id):
        top_n_iid[iid].append((uid, rating))        

  # Then sort the predictions for each user and retrieve the k highest ones.
  for iid, user_ratings in top_n_iid.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
  result = [iid for (iid, _) in user_ratings][:n]
  return result  

def predict_user_item(model, user_id, item_id):
  result = []
  # Print the recommended items for each user
  for uid, user_ratings in model.items():
    if str(uid) == str(user_id):
      for (iid, rating) in user_ratings:
        if str(iid) == str(item_id):
          result.append(rating)
  return result


## Training

In [ ]:
# part 1 - training
csv_filename = 'casasbahia.csv'
print('training model...')
model = train_model(csv_filename)
print('saving model...')
model_filename = save_model(model)
print(f'model saved to {model_filename}')

training model...
Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
model trained in 26.1 seconds
saving model...
tamanho do modelo: 36984 bytes
tamanho do arquivo: 62238699 bytes
model saved to 5e7c80d0-9c4f-400c-a789-b3446b661c01.model


## Prediction

In [ ]:
# part 2 - predicting
print('loading saved model from disk...')
model = load_model(model_filename)
user_id = 196
print(f'predicting user {user_id}..')
result_user = predict_user(model, user_id, n=5)
print(f'prediction result={result_user}')
if len(result_user) > 0:
  item_id = result_user[0]
  print(f'predicting item {item_id}..')
  result_item = predict_item(model, item_id, n=5)
  print(f'prediction result={result_item}')
  if len(result_item) > 0:
    print(f'predicting item {item_id} for user {user_id}..')
    result_user_item = predict_user_item(model, user_id, item_id)
    print(f'prediction result={result_user_item}')  

loading saved model from disk...
Carregando o modelo
Modelo carregado
predicting user 196..
prediction result=['98', '64', '408', '318', '483']
predicting item 98..
prediction result=['157', '173', '4', '252', '261']
predicting item 98 for user 196..
prediction result=[4.6788753121780635]
